# Preparation

This code is meant to be used during Lesson 2.5 of Andrew Pinkham's _Building backend web applications and APIs with Django_ class. 

To re-run this code you need a working *empty* database. The results of this notebook may vary depending on where in the git history you have navigated to, and whether you have made your own changes to the code.

Consider that if you have an SQLite3 or PostgreSQL database with data in it, running this code will modify the data.

In [1]:
from datetime import date

from blog.models import Post
from organizer.models import Tag, Startup, NewsLink

# Basic Interaction with Django Models

In [2]:
edut = Tag(name='Education', slug='education')

In [3]:
edut

<Tag: Education>

In [4]:
edut.name

'Education'

In [5]:
edut.save()  # saves the data in the model to the database!

In [6]:
edut.delete()  # deleted the data from the database!

(1, {'blog.Post_tags': 0, 'organizer.Startup_tags': 0, 'organizer.Tag': 1})

In [7]:
edut  # still in memory!

<Tag: Education>

## Creation and Destruction with Managers

In [8]:
type(Tag.objects)  # a model manager

django.db.models.manager.Manager

In [9]:
Tag.objects.create(name='Video Games', slug='video-games')

<Tag: Video Games>

In [10]:
# create multiple objects in a go!
Tag.objects.bulk_create([
    Tag(name='Django', slug='django'),
    Tag(name='Mobile', slug='mobile'),
    Tag(name='Web', slug='web'),
])

[<Tag: Django>, <Tag: Mobile>, <Tag: Web>]

In [11]:
Tag.objects.all()

<QuerySet [<Tag: Django>, <Tag: Mobile>, <Tag: Video Games>, <Tag: Web>]>

In [12]:
Tag.objects.all()[0]  # acts like a list

<Tag: Django>

In [13]:
type(Tag.objects.all())  # is not a list

django.db.models.query.QuerySet

In [14]:
# managers are not accessible to model instances, only to model classes!
try:
    edut.objects
except AttributeError as e:
    print(e)

Manager isn't accessible via Tag instances


## Methods of Data Retrieval

In [15]:
Tag.objects.all()

<QuerySet [<Tag: Django>, <Tag: Mobile>, <Tag: Video Games>, <Tag: Web>]>

In [16]:
Tag.objects.count()

4

### The `get` method

In [17]:
Tag.objects.get(slug='django')

<Tag: Django>

In [18]:
type(Tag.objects.all())

django.db.models.query.QuerySet

In [19]:
type(Tag.objects.get(slug='django'))

organizer.models.Tag

In [20]:
# case-sensitive!
try:
    Tag.objects.get(slug='Django')
except Tag.DoesNotExist as e:
    print(e)

Tag matching query does not exist.


In [21]:
# the i is for case-Insensitive
Tag.objects.get(slug__iexact='DJANGO')

<Tag: Django>

In [22]:
Tag.objects.get(slug__istartswith='DJ')

<Tag: Django>

In [23]:
Tag.objects.get(slug__contains='an')

<Tag: Django>

In [24]:
# get always returns a single object
try:
    # djangO, mObile, videO-games
    Tag.objects.get(slug__contains='o')
except Tag.MultipleObjectsReturned as e:
    print(e)

get() returned more than one Tag -- it returned 3!


### The `filter` method

In [25]:
## unlike get, can fetch multiple objects
Tag.objects.filter(slug__contains='o')

<QuerySet [<Tag: Django>, <Tag: Mobile>, <Tag: Video Games>]>

In [26]:
type(Tag.objects.filter(slug__contains='o'))

django.db.models.query.QuerySet

### Chaining Calls

In [27]:
Tag.objects.filter(slug__contains='o').order_by('-name')

<QuerySet [<Tag: Video Games>, <Tag: Mobile>, <Tag: Django>]>

In [28]:
# first we call order_by on the manager
Tag.objects.order_by('-name')

<QuerySet [<Tag: Web>, <Tag: Video Games>, <Tag: Mobile>, <Tag: Django>]>

In [29]:
# now we call filter on the manager, and order the resulting queryset
Tag.objects.filter(slug__contains='e').order_by('-name')

<QuerySet [<Tag: Web>, <Tag: Video Games>, <Tag: Mobile>]>

### `values` and `values_list`

In [30]:
Tag.objects.values_list()

<QuerySet [(3, 'Django', 'django'), (4, 'Mobile', 'mobile'), (2, 'Video Games', 'video-games'), (5, 'Web', 'web')]>

In [31]:
type(Tag.objects.values_list())

django.db.models.query.QuerySet

In [32]:
Tag.objects.values_list('name', 'slug')

<QuerySet [('Django', 'django'), ('Mobile', 'mobile'), ('Video Games', 'video-games'), ('Web', 'web')]>

In [33]:
Tag.objects.values_list('name')

<QuerySet [('Django',), ('Mobile',), ('Video Games',), ('Web',)]>

In [34]:
Tag.objects.values_list('name', flat=True)

<QuerySet ['Django', 'Mobile', 'Video Games', 'Web']>

In [35]:
type(Tag.objects.values_list('name', flat=True))

django.db.models.query.QuerySet

## Data in Memory vs Data in the Database

In [36]:
jb = Startup.objects.create(
    name='JamBon Software',
    slug='jambon-software',
    contact='django@jambonsw.com',
    description='Web and Mobile Consulting.\n'
                'Django Tutoring.\n',
    founded_date=date(2013, 1, 18),
    website='https://jambonsw.com/',
)
jb  # this output only clear because of __str__()

<Startup: JamBon Software>

In [37]:
jb.founded_date

datetime.date(2013, 1, 18)

In [38]:
jb.founded_date = date(2014,1,1)
# we're not calling save() !
jb.founded_date

datetime.date(2014, 1, 1)

In [39]:
# get version in database
jb = Startup.objects.get(slug='jambon-software')
# work above is all for nought because we didn't save()
jb.founded_date

datetime.date(2013, 1, 18)

## Connecting Data through Relations

In [40]:
djt = Post.objects.create(
    title='Django Training',
    slug='django-training',
    text=(
        "Learn Django in a classroom setting "
        "with JamBon Software."),
)
djt

<Post: Django Training on 2018-08-01>

In [41]:
djt.pub_date = date(2013, 1, 18)
djt.save()
djt

<Post: Django Training on 2013-01-18>

In [42]:
type(djt.tags)

django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager

In [43]:
type(djt.startups)

django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager

In [44]:
djt.tags.all()

<QuerySet []>

In [45]:
djt.startups.all()

<QuerySet []>

In [46]:
django = Tag.objects.get(slug__contains='django')
djt.tags.add(django)
djt.tags.all()

<QuerySet [<Tag: Django>]>

In [47]:
django.blog_posts.all()  # a "reverse" relation

<QuerySet [<Post: Django Training on 2013-01-18>]>

In [48]:
django.startup_set.add(jb)  # a "reverse" relation
django.startup_set.all()

<QuerySet [<Startup: JamBon Software>]>

In [49]:
jb.tags.all()  # the "forward" relation

<QuerySet [<Tag: Django>]>

In [50]:
# on more time, for repetition!
djt

<Post: Django Training on 2013-01-18>

In [51]:
# "forward" relation
djt.startups.add(jb)
djt.startups.all()

<QuerySet [<Startup: JamBon Software>]>

In [52]:
jb.blog_posts.all()  # "reverse" relation

<QuerySet [<Post: Django Training on 2013-01-18>]>